# Working with Design Reviews

This example shows how you can work with Design Reviews on AllSpice Hub using py-allspice.

First, let's begin with the usual setup:

In [ ]:
from allspice import AllSpice, Repository, DesignReview

allspice_client = AllSpice(token_text="<YOUR TOKEN HERE>")

## Creating, Fetching and Editing Design Reviews

Design Reviews are made on repositories, so let's start with getting one.

In [ ]:
# You can change the strings here to fit your organization/user and repository name.
repo = Repository.request(allspice_client, "myorg", "amazingpcb")

To create a design review, you need to know the "base" and "head" branches. The "base" branch is
the branch *to which* you want to merge. Typically, this will be the `main` branch. The "head"
branch is the branch *from which* you are merging. This can be a feature branch, such as
`add-pcblib`.

In [ ]:
# Create a Design Review

dr = repo.create_design_review(
    title="Add PCB Library",
    head="add-pcblib",
    base="main",
)

print(f"New Design Review created: #{dr.number} - {dr.title}.")


There are other options you can use to conveniently assign users or set a due date or milestone.

In [ ]:
import datetime

dr = repo.create_design_review(
    title="Add PCB Library",
    head="add-pcblib",
    base="main",
    assignees={"shrik450"},
    body="This DR adds a pcblib to our awesome PCB!",
    # 7 days from now
    due_date=datetime.datetime.now() + datetime.timedelta(days=7),
    milestone="1",
)

print(f"New Design Review created: #{dr.number} - {dr.title}.")
print(dr.body)
print(f"Assigned to {dr.assignees}, with due date {dr.due_date}.")

All of these parameters are optional. Only the `title`, `base` and `head` params are required.

You can also get all existing Design Reviews on a repository:

In [ ]:
drs = repo.get_design_reviews()

for dr in drs:
    print(f"#{dr.number} - {dr.title}.")

This allows you to filter the DRs, too:

In [ ]:
drs = repo.get_design_reviews(state="open", milestone="1", labels="important")

Each filter is optional, so you can pick which ones you want.

You can also get an individual DR if you know its number:

In [ ]:
dr = DesignReview.request(allspice_client, "myorg", "amazingpcb", "3")

To edit a DR, simply change its fields and call `commit()`:

In [ ]:
dr.title = "Add Amazing PCB Library"
dr.body = "This DR adds an *amazing* pcblib to our awesome PCB!"
dr.commit()

Note that not all fields of the DR can be changed. This is the current list:

- allow_maintainer_edits
- assignee
- assignees
- base
- body
- due_date
- milestone
- state
- title

This list may change, so you can refer to https://hub.allspice.io/api/swagger#/repository/repoEditPullRequest
for an always up-to-date list.

DRs can't be deleted via py-allspice, but you can set their state to closed.

## Fetching and Adding Comments from/to DRs

DRs can have comments, just like issues. In this section, comments refer to top-level comments that
appear in the "Conversation" tab, and are not tied to a specific review.

In [ ]:
# This returns all comments
comments = dr.get_comments()

for comment in comments:
    print(f"Comment by {comment.user.username}:\n{comment.body}")

You can add a comment to the DR:

In [ ]:
comment = dr.create_comment("This PCB Lib is pretty awesome!")

print(f"Comment by {comment.user.username}:\n{comment.body}")

# Putting Everything Together

Using the concepts in this guide, here's a small script that gets all the DRs on a repo, checks if
any of them don't have an assignee, and makes a comment about it:

In [ ]:
repo_owner = "myorg"
repo_name = "amazingpcb"

repo = Repository.request(allspice_client, repo_owner, repo_name)

drs = repo.get_design_reviews(state=DesignReview.OPEN)

for dr in drs:
    if dr.assignees is None or len(dr.assignees) == 0:
        dr.create_comment(f"""
        This design review has no assignee!

        @{dr.user.username} - please assign this DR to an owner, or assign it to yourself.
        """)
